In [3]:
import pandas as pd
import os, sys

In [5]:
from stadincijfers import stadincijfers

In [6]:
sic = stadincijfers("antwerpen")

In [4]:
vars = sic.odataVariables(450, 500)
vars_s = pd.Series(vars)
vars_s.to_csv("data/vars.csv")
vars_s.tail()

reading data, lines 450 to 500 this can take a while
450 460 470 480 490 500 

br_industrie_oproep                                       oproep brandbestrijding - industrie
br_industrie_p                                                 brandbestrijding - % industrie
br_industrie_p_excl                                            brandbestrijding - % industrie
br_melding_van_brandmeld_centrale           brandbestrijding - melding van brandmeld centrale
br_melding_van_brandmeld_centrale_oproep    oproep brandbestrijding - melding van brandmel...
dtype: object

In [7]:
# get data as a json object convert to dict
v = sic.selectiontableasjson('autodeelplaatsen_cambio', geolevel='sector', periodlevel='year', period=None )
v['rows'][0:10]

[['12MQ', '2020', ''],
 ['A00-', '2020', '2'],
 ['A01-', '2020', ''],
 ['A02-', '2020', '9'],
 ['A03-', '2020', ''],
 ['A04-', '2020', '3'],
 ['A05-', '2020', '2'],
 ['A081', '2020', ''],
 ['A10-', '2020', ''],
 ['A11-', '2020', '5']]

In [21]:
cambio2016 = sic.selectiontableasDataframe('autodeelplaatsen_cambio', geolevel='wijk', periodlevel='year', period=2016)
cambio2016.head(10)


,Geo,Perioden,# autodeelplaatsen Cambio
0,Amandus - Atheneum,2016,2.0
1,Brederode,2016,NaN
2,Centraal Station,2016,NaN
3,Dam,2016,NaN
4,Eilandje,2016,2.0
5,Haringrode,2016,2.0
6,Harmonie,2016,7.0
7,Haven Antwerpen,2016,NaN
8,Historisch Centrum,2016,4.0
9,Kiel,2016,NaN


In [20]:
cambioNu = sic.selectiontableasDataframe('autodeelplaatsen_cambio', geolevel='wijk', periodlevel='year', period=2020)
"cambios staansplaatsen in 2016 {} en nu {}".format( cambio2016['# autodeelplaatsen Cambio'].sum() ,
                                            cambioNu['# autodeelplaatsen Cambio'].sum() ) 

'cambios in 2016 74.0 en nu 156.0'